## Compare results for multiple cosmologies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_3_different_universes_6k/'
f_strg=parent_dir+'*.npy'
f_list=glob.glob(f_strg)
print(len(f_list))

27


In [6]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1

In [7]:
### Extract fixed univ data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:8000][:,0,:,:]
print(s_val.shape)

(8000, 128, 128)


In [8]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [9]:
# print(bins)

In [10]:
df_runs=pd.DataFrame([])
dict1={}
for count,fname in enumerate(f_list):
    lst=fname.split('.npy')[0].split('/')[-1].split('_')
#     print(lst)
    keys=['omega_m','sigma_8','H_0']
    values=[float(lst[0][2:]),float(lst[1][2:]),float(lst[2][1:])]
    dict1=dict(zip(keys,values))
    strg=str(count)+'_Og={0}_Sg={1}_H0={2}'.format(dict1['omega_m'],dict1['sigma_8'],dict1['H_0'])
    print(strg)

    dict1['label']=strg
    dict1['fname']=fname
    images=f_transform(np.load(fname)[:,:,:,0])
    dict1['num_img']=images.shape[0]
    f_transform(images)
#     print(images.shape,np.max(images),np.min(images))
    dict_sample=f_compute_hist_spect(images,bins) ## list of 5 numpy arrays 
    dict1.update(dict_sample)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)
    
df_runs

0_Og=0.3_Sg=1.1_H0=70.0
1_Og=0.45_Sg=0.5_H0=70.0
2_Og=0.45_Sg=0.8_H0=100.0
3_Og=0.3_Sg=0.5_H0=40.0
4_Og=0.45_Sg=0.5_H0=40.0
5_Og=0.45_Sg=0.5_H0=100.0
6_Og=0.3_Sg=1.1_H0=40.0
7_Og=0.45_Sg=1.1_H0=70.0
8_Og=0.45_Sg=0.8_H0=40.0
9_Og=0.15_Sg=0.8_H0=70.0
10_Og=0.15_Sg=0.5_H0=70.0
11_Og=0.3_Sg=0.8_H0=100.0
12_Og=0.45_Sg=1.1_H0=100.0
13_Og=0.45_Sg=0.8_H0=70.0
14_Og=0.3_Sg=1.1_H0=100.0
15_Og=0.15_Sg=1.1_H0=70.0
16_Og=0.15_Sg=1.1_H0=40.0
17_Og=0.3_Sg=0.5_H0=70.0
18_Og=0.3_Sg=0.5_H0=100.0
19_Og=0.15_Sg=0.8_H0=40.0
20_Og=0.3_Sg=0.8_H0=70.0
21_Og=0.15_Sg=0.5_H0=40.0
22_Og=0.15_Sg=0.8_H0=100.0
23_Og=0.45_Sg=1.1_H0=40.0
24_Og=0.15_Sg=1.1_H0=100.0
25_Og=0.3_Sg=0.8_H0=40.0
26_Og=0.15_Sg=0.5_H0=100.0


,H_0,fname,hist_bin_centers,hist_err,hist_val,label,num_img,omega_m,sigma_8,spec_err,spec_val
0,70.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.001794186519672734, 0.0016756928974251588, ...","[1.2743357452837472, 0.7571865331869473, 0.212...",0_Og=0.3_Sg=1.1_H0=70.0,12288.0,0.30,1.1,"[82.1838118048652, 63.087633059874285, 52.0777...","[68.513382789837, 57.997503993345795, 49.60497..."
1,70.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002101543081766953, 0.0022851389601648326, ...","[0.9119968437830351, 1.225620390872648, 0.3329...",1_Og=0.45_Sg=0.5_H0=70.0,6144.0,0.45,0.5,"[46.37957370356237, 39.054943748905956, 32.756...","[46.59129939068625, 42.464904240789956, 37.509..."
2,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002057638519924891, 0.0021155861699592325, ...","[1.1130545168022479, 0.9423240437718244, 0.273...",2_Og=0.45_Sg=0.8_H0=100.0,6144.0,0.45,0.8,"[50.622706191432464, 44.304512791857405, 40.42...","[50.34933765660355, 47.325373831140496, 43.769..."
3,40.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0026673505222881335, 0.0026575581467473274,...","[0.9042813801248238, 1.2675558348154137, 0.312...",3_Og=0.3_Sg=0.5_H0=40.0,6144.0,0.30,0.5,"[59.64705994353596, 44.6356861730504, 36.49935...","[55.008585987309935, 44.79453156053694, 37.697..."
4,40.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0025283401904641865, 0.0025394311931473852,...","[0.9760136629484015, 1.1685315388559872, 0.291...",4_Og=0.45_Sg=0.5_H0=40.0,6144.0,0.45,0.5,"[66.92083289431623, 47.09752907948722, 37.6989...","[57.747397258306705, 46.674927406153756, 39.14..."
5,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00191988799161104, 0.002157477544797874, 0....","[0.8501529715532047, 1.285632135464296, 0.3741...",5_Og=0.45_Sg=0.5_H0=100.0,6144.0,0.45,0.5,"[34.277809729556026, 31.350043758751234, 28.19...","[38.23370080480413, 36.49546481101135, 34.1241..."
6,40.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002983663198749289, 0.002661954005542851, 0...","[1.335513714420416, 0.6938585053641709, 0.1888...",6_Og=0.3_Sg=1.1_H0=40.0,6144.0,0.30,1.1,"[111.60398257096463, 78.52617148898288, 57.830...","[78.18431304493674, 60.27149575698096, 48.6244..."
7,70.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002371895026886743, 0.0022429183783147506, ...","[1.309027794399709, 0.7069190751474749, 0.2052...",7_Og=0.45_Sg=1.1_H0=70.0,6144.0,0.45,1.1,"[84.08914398490393, 64.7223603858224, 53.40638...","[68.80084693309082, 59.76103712867192, 51.3981..."
8,40.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002670132436164184, 0.0025796172257118648, ...","[1.2336380513539948, 0.8244996083562511, 0.215...",8_Og=0.45_Sg=0.8_H0=40.0,6144.0,0.45,0.8,"[89.17781030592538, 64.15112020167501, 50.8912...","[69.24798217070713, 56.05898460123402, 47.5246..."
9,70.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002631181653444825, 0.002546800593352497, 0...","[1.033426525874199, 1.0899478213887535, 0.2745...",9_Og=0.15_Sg=0.8_H0=70.0,6144.0,0.15,0.8,"[69.99140831295139, 52.75348763429585, 43.1799...","[60.7026626039771, 49.33742389361722, 41.81874..."


In [11]:
# df_runs

In [12]:
# ### Add some external runs

# ### Extract some LBANN training run data

# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200725_204329_batchsize_256/dump_outs/trainer0/model0/'
# lst=[(17, 14207), (16, 13380), (18, 14922), (16, 13089)]
# prefix='lbann_train_'

# for count,run in enumerate(lst[:1]):
#     epoch,step=run[0],run[1]
#     key=prefix+'{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
# #     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
# #     dict1.update(dict_chisqrs)
#     del(images)
# #     df_runs=df_runs.append(dict1,ignore_index=True)



In [13]:

# def f_plot_hist_spec_best_combined(df,dict_bkg):

#     fig=plt.figure(figsize=(6,6))
#     ax1=fig.add_subplot(121)
#     ax2=fig.add_subplot(122)
#     for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

#         x1=row.hist_bin_centers
#         y1=row.hist_val
#         yerr1=row.hist_err
#         x1=f_invtransform(x1)

#         y2=row.spec_val
#         yerr2=row.spec_err
#         x2=np.arange(len(y2))

#         label=row.label
#         ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
#     #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

#         ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
#         ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

#     ### Plot input data
#     x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
#     x=f_invtransform(x)
#     ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

#     y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
#     x=np.arange(len(y))
#     ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

# #     plt.legend()
#     # plt.yscale('log')
#     ax1.set_xscale('symlog',linthreshx=50)
#     ax1.set_yscale('log')
#     ax2.set_yscale('log')


# f_plot_hist_spec_best(df_runs,dict_val)
# # f_plot_hist_spec_best_combined(df_runs,dict1)

In [14]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
        
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')    
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')




In [21]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

# interact_manual(f_widget_compare,df=fixed(df_runs[(df_runs.H_0==70.0)&(df_runs.omega_m==0.3)]),
interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))


interactive(children=(SelectMultiple(description='labels_list', options=('0_Og=0.3_Sg=1.1_H0=70.0', '1_Og=0.45…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [16]:
for i in df_runs.loc[[5,22,24,26],:].label.values:
    print(i)

5_Og=0.45_Sg=0.5_H0=100.0
22_Og=0.15_Sg=0.8_H0=100.0
24_Og=0.15_Sg=1.1_H0=100.0
26_Og=0.15_Sg=0.5_H0=100.0


In [17]:
df_runs[df_runs.H_0==100.0]

,H_0,fname,hist_bin_centers,hist_err,hist_val,label,num_img,omega_m,sigma_8,spec_err,spec_val
2,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002057638519924891, 0.0021155861699592325, ...","[1.1130545168022479, 0.9423240437718244, 0.273...",2_Og=0.45_Sg=0.8_H0=100.0,6144.0,0.45,0.8,"[50.622706191432464, 44.304512791857405, 40.42...","[50.34933765660355, 47.325373831140496, 43.769..."
5,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00191988799161104, 0.002157477544797874, 0....","[0.8501529715532047, 1.285632135464296, 0.3741...",5_Og=0.45_Sg=0.5_H0=100.0,6144.0,0.45,0.5,"[34.277809729556026, 31.350043758751234, 28.19...","[38.23370080480413, 36.49546481101135, 34.1241..."
11,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0021517350427635276, 0.002253705852467284, ...","[1.0583703545213015, 1.0206363216863399, 0.290...",11_Og=0.3_Sg=0.8_H0=100.0,6144.0,0.30,0.8,"[50.660219361866496, 44.53145913171041, 38.008...","[51.19487078686936, 46.76870813182048, 42.4743..."
12,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002167039888439183, 0.002145257531539029, 0...","[1.2496022018239468, 0.766804934729164, 0.2288...",12_Og=0.45_Sg=1.1_H0=100.0,6144.0,0.45,1.1,"[63.8597759487539, 52.59402600159309, 45.53167...","[57.97472787728825, 53.367744880826706, 49.412..."
14,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0022630228056118684, 0.002223094377403179, ...","[1.2205978663456871, 0.8178355706415162, 0.232...",14_Og=0.3_Sg=1.1_H0=100.0,6144.0,0.30,1.1,"[61.38965895003737, 51.91439376375957, 45.0655...","[57.88362549704511, 52.66462544975627, 46.8336..."
18,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0019549641138334175, 0.0022084054726393115,...","[0.7907273789774396, 1.3811827919301376, 0.386...",18_Og=0.3_Sg=0.5_H0=100.0,6144.0,0.30,0.5,"[34.29757162728287, 30.409143446035163, 28.357...","[39.0143284549971, 35.6791402007112, 32.943312..."
22,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0024126918375610737, 0.002479253536807315, ...","[0.9822396065298299, 1.1481775063081645, 0.299...",22_Og=0.15_Sg=0.8_H0=100.0,6144.0,0.15,0.8,"[56.91750954606087, 46.03386217096141, 36.7174...","[53.789939106689, 45.79390656184867, 39.285175..."
24,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0025121237332078323, 0.0024631773746304183,...","[1.1633352071251146, 0.9028787626823127, 0.245...",24_Og=0.15_Sg=1.1_H0=100.0,6144.0,0.15,1.1,"[73.08220472694502, 59.185651550049386, 48.656...","[62.30634150974515, 53.980937433486815, 47.119..."
26,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002111239338762676, 0.00236280660196559, 0....","[0.6924190539100241, 1.5380591177790697, 0.411...",26_Og=0.15_Sg=0.5_H0=100.0,6144.0,0.15,0.5,"[37.17101013506972, 30.46360552420539, 25.6140...","[39.84291121325191, 34.30002820842598, 29.9148..."


In [18]:
df_runs[(df_runs.H_0==100.0) & (df_runs.omega_m==0.15)]

,H_0,fname,hist_bin_centers,hist_err,hist_val,label,num_img,omega_m,sigma_8,spec_err,spec_val
22,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0024126918375610737, 0.002479253536807315, ...","[0.9822396065298299, 1.1481775063081645, 0.299...",22_Og=0.15_Sg=0.8_H0=100.0,6144.0,0.15,0.8,"[56.91750954606087, 46.03386217096141, 36.7174...","[53.789939106689, 45.79390656184867, 39.285175..."
24,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0025121237332078323, 0.0024631773746304183,...","[1.1633352071251146, 0.9028787626823127, 0.245...",24_Og=0.15_Sg=1.1_H0=100.0,6144.0,0.15,1.1,"[73.08220472694502, 59.185651550049386, 48.656...","[62.30634150974515, 53.980937433486815, 47.119..."
26,100.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002111239338762676, 0.00236280660196559, 0....","[0.6924190539100241, 1.5380591177790697, 0.411...",26_Og=0.15_Sg=0.5_H0=100.0,6144.0,0.15,0.5,"[37.17101013506972, 30.46360552420539, 25.6140...","[39.84291121325191, 34.30002820842598, 29.9148..."


In [19]:
df_runs.label

0        0_Og=0.3_Sg=1.1_H0=70.0
1       1_Og=0.45_Sg=0.5_H0=70.0
2      2_Og=0.45_Sg=0.8_H0=100.0
3        3_Og=0.3_Sg=0.5_H0=40.0
4       4_Og=0.45_Sg=0.5_H0=40.0
5      5_Og=0.45_Sg=0.5_H0=100.0
6        6_Og=0.3_Sg=1.1_H0=40.0
7       7_Og=0.45_Sg=1.1_H0=70.0
8       8_Og=0.45_Sg=0.8_H0=40.0
9       9_Og=0.15_Sg=0.8_H0=70.0
10     10_Og=0.15_Sg=0.5_H0=70.0
11     11_Og=0.3_Sg=0.8_H0=100.0
12    12_Og=0.45_Sg=1.1_H0=100.0
13     13_Og=0.45_Sg=0.8_H0=70.0
14     14_Og=0.3_Sg=1.1_H0=100.0
15     15_Og=0.15_Sg=1.1_H0=70.0
16     16_Og=0.15_Sg=1.1_H0=40.0
17      17_Og=0.3_Sg=0.5_H0=70.0
18     18_Og=0.3_Sg=0.5_H0=100.0
19     19_Og=0.15_Sg=0.8_H0=40.0
20      20_Og=0.3_Sg=0.8_H0=70.0
21     21_Og=0.15_Sg=0.5_H0=40.0
22    22_Og=0.15_Sg=0.8_H0=100.0
23     23_Og=0.45_Sg=1.1_H0=40.0
24    24_Og=0.15_Sg=1.1_H0=100.0
25      25_Og=0.3_Sg=0.8_H0=40.0
26    26_Og=0.15_Sg=0.5_H0=100.0
Name: label, dtype: object

In [27]:
np.unique(df_runs.sigma_8.values)

array([0.5, 0.8, 1.1])